_한국어로 기계번역됨_


# 노드 간의 개인 상태 전달 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 이해가 필요합니다:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/low_level/#multiple-schemas">
                    여러 스키마
                </a>
            </li>
        </ul>
    </p>
</div> 

어떤 경우에는 노드가 중간 논리에 중요한 정보를 교환하도록 하고 싶지만, 이 정보는 그래프의 주요 스키마의 일부일 필요는 없습니다. 이 개인 데이터는 그래프의 전체 입력/출력과는 관련이 없으며 특정 노드 간에만 공유되어야 합니다.

이 사용 방법 가이드에서는 세 개의 노드(노드_1, 노드_2, 노드_3)로 구성된 예제 순차 그래프를 만들고, 개인 데이터가 첫 번째 두 단계(노드_1과 노드_2) 간에 전달되고, 세 번째 단계(노드_3)는 공용 전체 상태에만 접근할 수 있도록 합니다.

## 설정

먼저, 필요한 패키지를 설치해 보겠습니다.


In [1]:
%%capture --no-stderr
%pip install -U langgraph


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 개선하세요. LangSmith는 LangGraph로 구축된 LLM 앱을 디버깅, 테스트 및 모니터링하기 위해 추적 데이터를 사용할 수 있게 해줍니다 — 시작하는 방법에 대해 더 읽으려면 <a href="https://docs.smith.langchain.com">여기</a>를 클릭하세요.
    </p>
</div>


## 그래프 정의 및 사용


In [1]:
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict


# The overall state of the graph (this is the public state shared across nodes)
class OverallState(TypedDict):
    a: str


# Output from node_1 contains private data that is not part of the overall state
class Node1Output(TypedDict):
    private_data: str


# The private data is only shared between node_1 and node_2
def node_1(state: OverallState) -> Node1Output:
    output = {"private_data": "set by node_1"}
    print(f"Entered node `node_1`:\n\tInput: {state}.\n\tReturned: {output}")
    return output


# Node 2 input only requests the private data available after node_1
class Node2Input(TypedDict):
    private_data: str


def node_2(state: Node2Input) -> OverallState:
    output = {"a": "set by node_2"}
    print(f"Entered node `node_2`:\n\tInput: {state}.\n\tReturned: {output}")
    return output


# Node 3 only has access to the overall state (no access to private data from node_1)
def node_3(state: OverallState) -> OverallState:
    output = {"a": "set by node_3"}
    print(f"Entered node `node_3`:\n\tInput: {state}.\n\tReturned: {output}")
    return output


# Build the state graph
builder = StateGraph(OverallState)
builder.add_node(node_1)  # node_1 is the first node
builder.add_node(
    node_2
)  # node_2 is the second node and accepts private data from node_1
builder.add_node(node_3)  # node_3 is the third node and does not see the private data
builder.add_edge(START, "node_1")  # Start the graph with node_1
builder.add_edge("node_1", "node_2")  # Pass from node_1 to node_2
builder.add_edge(
    "node_2", "node_3"
)  # Pass from node_2 to node_3 (only overall state is shared)
builder.add_edge("node_3", END)  # End the graph after node_3
graph = builder.compile()

# Invoke the graph with the initial state
response = graph.invoke(
    {
        "a": "set at start",
    }
)

print()
print(f"Output of graph invocation: {response}")


Entered node `node_1`:
	Input: {'a': 'set at start'}.
	Returned: {'private_data': 'set by node_1'}
Entered node `node_2`:
	Input: {'private_data': 'set by node_1'}.
	Returned: {'a': 'set by node_2'}
Entered node `node_3`:
	Input: {'a': 'set by node_2'}.
	Returned: {'a': 'set by node_3'}

Output of graph invocation: {'a': 'set by node_3'}
